In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import RegularGridInterpolator
from scipy.spatial.distance import euclidean
from tqdm import tqdm
from conehead.geometry import (
    global_to_beam, line_block_plane_collision, line_calc_limit_plane_collision
)
from conehead.kernel import PolyenergeticKernel
# from conehead.dda_3d_c import dda_3d_c
import conehead.nist
from conehead.source import Source
from conehead.block import Block
from conehead.phantom import SimplePhantom
from conehead.conehead import Conehead

In [3]:
# Choose source
source = Source("varian_clinac_6MV")
source.gantry(0)
source.collimator(0)

# Create 10 cm x 10 cm collimator opening
block = Block(source.rotation)
block.set_square(10)

# Simple phantom
phantom = SimplePhantom()

# Calculation settings
settings = {
    'stepSize': 0.1,  # Stepsize when raytracing effective depth
    'sPri': 1.0,  # Primary source strength (photons/mm^2)
    'softRatio': 0.0025,  # mm^-1
    'softLimit': 20,  # cm
    'eLow': 0.01,  # MeV
    'eHigh': 7.0,  # MeV
    'eNum': 500,  # Spectrum samples
}

In [4]:
# Transform phantom to beam coords
print("Transforming phantom to beam coords...")
phantom_beam = np.zeros_like(phantom.positions)
_, xlen, ylen, zlen = phantom_beam.shape
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            phantom_beam[:, x, y, z] = global_to_beam(
                phantom.positions[:, x, y, z],
                source.position,
                source.rotation
            )

print("Interpolating phantom densities...")
phantom_densities_interp = RegularGridInterpolator(
    (phantom_beam[0, :, 0, 0],
     phantom_beam[1, 0, :, 0],
     phantom_beam[2, 0, 0, :]),
    phantom.densities,
    method='nearest',
    bounds_error=False,
    fill_value=0
)

# Create dose grid (just the same size as the phantom for now)
dose_grid_positions = np.copy(phantom_beam)
dose_grid_dim = np.array([2, 2, 2], dtype=np.float64)  # cm

# Perform hit testing to find which dose grid voxels are in the beam
print("Performing hit-testing of dose grid voxels...")
_, xlen, ylen, zlen = dose_grid_positions.shape
dose_grid_blocked = np.zeros((xlen, ylen, zlen))
dose_grid_OAD = np.zeros((xlen, ylen, zlen))
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            voxel = dose_grid_positions[:, x, y, z]
            psi = line_block_plane_collision(voxel)
            dose_grid_blocked[x, y, z] = (
                block.block_values_interp([psi[0], psi[1]])
            )
            # Save off-axis distance (at iso plane) for later
            dose_grid_OAD[x, y, z] = (
                euclidean(np.array([0, 0, source.SAD]), psi)
            )

# Calculate effective depths of dose grid voxels
print("Calculating effective depths of dose grid voxels...")
dose_grid_d_eff = np.zeros_like(dose_grid_blocked)
xlen, ylen, zlen = dose_grid_d_eff.shape
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            voxel = dose_grid_positions[:, x, y, z]
            psi = line_calc_limit_plane_collision(voxel)
            dist = np.sqrt(np.sum(np.power(voxel - psi, 2)))
            num_steps = np.floor(dist / settings['stepSize'])
            xcoords = np.linspace(voxel[0], psi[0], num_steps)
            ycoords = np.linspace(voxel[1], psi[1], num_steps)
            zcoords = np.linspace(voxel[2], psi[2], num_steps)
            dose_grid_d_eff[x, y, z] = np.sum(
                phantom_densities_interp(
                    np.dstack((xcoords, ycoords, zcoords))
                ) * settings['stepSize']
            )

# Calculate photon fluence at dose grid voxels
print("Calculating fluence...")
dose_grid_fluence = np.zeros_like(dose_grid_blocked)
xlen, ylen, zlen = dose_grid_fluence.shape
dose_grid_fluence = (
    settings['sPri'] * -source.SAD /
    dose_grid_positions[2, :, :, :] *
    dose_grid_blocked
)

# Calculate beam softening factor for dose grid voxels
print("Calculating beam softening factor...")
f_soften = np.ones_like(dose_grid_OAD)
f_soften[dose_grid_OAD < settings['softLimit']] = 1 / (
    1 - settings['softRatio'] *
    dose_grid_OAD[dose_grid_OAD < settings['softLimit']]
)

# Calculate TERMA of dose grid voxels
print("Calculating TERMA...")
E = np.linspace(settings['eLow'], settings['eHigh'], settings['eNum'])
spectrum_weights = source.weights(E)
mu_water = conehead.nist.mu_water(E)
dose_grid_terma = np.zeros_like(dose_grid_blocked)
xlen, ylen, zlen = dose_grid_terma.shape
for x in tqdm(range(xlen)):
    for y in range(ylen):
        for z in range(zlen):
            dose_grid_terma[x, y, z] = (
                np.sum(
                    spectrum_weights *
                    dose_grid_fluence[x, y, z] *
                    np.exp(
                        -mu_water * f_soften[x, y, z] *
                        dose_grid_d_eff[x, y, z]
                    ) * E * mu_water
                )
            )


 14%|█▍        | 3/21 [00:00<00:00, 26.49it/s]

Transforming phantom to beam coords...


 10%|▉         | 2/21 [00:00<00:01, 14.17it/s]

Interpolating phantom densities...
Performing hit-testing of dose grid voxels...


  0%|          | 0/21 [00:00<?, ?it/s]/home/sam/projects/conehead/env/lib/python3.6/site-packages/ipykernel_launcher.py:58: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
/home/sam/projects/conehead/env/lib/python3.6/site-packages/ipykernel_launcher.py:59: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
/home/sam/projects/conehead/env/lib/python3.6/site-packages/ipykernel_launcher.py:60: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
  5%|▍         | 1/21 [00:00<00:03,  5.97it/s]

Calculating effective depths of dose grid voxels...


 43%|████▎     | 9/21 [00:00<00:00, 83.69it/s]

Calculating fluence...
Calculating beam softening factor...
Calculating TERMA...


100%|██████████| 21/21 [00:00<00:00, 83.68it/s]


In [49]:
# Calculate dose of dose grid voxels
print("Convolving kernel...")
kernel = PolyenergeticKernel()
dose_grid_dose = np.zeros_like(dose_grid_terma)
phis = [p for p in kernel.cumulative.keys() if p != "radii"]
thetas = np.linspace(0, 360, 6, endpoint=False)

Convolving kernel...


In [50]:
def convolve_python(dose_grid_terma, dose_grid_dose, dose_grid_dim, thetas, phis, kernel):

    xlen, ylen, zlen = dose_grid_terma.shape
    
    for x in tqdm(range(xlen)):
        for y in range(ylen):
            for z in range(zlen):
                T = dose_grid_terma[x, y, z]
                if T:
                    for theta in thetas:
                        for phi in phis:

                            # Raytracing
                            phi_rad = float(phi) * np.pi / 180
                            theta_rad = theta * np.pi / 180
                            direction = np.array([
                                np.cos(theta_rad) * np.sin(phi_rad),
                                np.sin(theta_rad) * np.sin(phi_rad),
                                np.cos(phi_rad)
                            ], dtype=np.float64)
                            direction /= np.sum(direction**2)  # Normalise
                            direction = np.around(  # discretise
                                direction,
                                decimals=6
                            )
                            intersections, voxels = dda_3d_c(
                                direction,
                                np.array(
                                    dose_grid_terma.shape,
                                    dtype=np.int32
                                ),
                                np.array([x, y, z], dtype=np.int32),
                                dose_grid_dim
                            )

                            intersections = np.array(
                                [int(x) for x in (intersections * 100 - 50)]
                            )
                            intersections = np.absolute(intersections)

                            for e, _ in enumerate(intersections):
                                v = voxels[e]
                                if e == 0:
                                    k = kernel.cumulative[phi][
                                        intersections[e]
                                    ]
                                else:
                                    k = kernel.cumulative[phi][
                                        intersections[e]
                                    ] - kernel.cumulative[phi][
                                        intersections[e - 1]
                                    ]
                                dose_grid_dose[v[0], v[1], v[2]] += T * k

In [53]:
%%time
convolve_python(dose_grid_terma, dose_grid_dose, dose_grid_dim, thetas, phis, kernel)

100%|██████████| 21/21 [00:26<00:00,  1.27s/it]

CPU times: user 26.6 s, sys: 220 ms, total: 26.8 s
Wall time: 26.8 s


In [25]:
load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [26]:
# Calculate dose of dose grid voxels
print("Convolving kernel...")
kernel = PolyenergeticKernel()
dose_grid_dose = np.zeros_like(dose_grid_terma, dtype=np.float64)
phis = np.array(sorted([float(p) for p in kernel.cumulative.keys() if p != "radii"]), dtype=np.float64)
thetas = np.linspace(0, 360, 6, endpoint=False, dtype=np.float64)

Convolving kernel...


In [1]:
%%cython

import numpy as np
cimport cython
cimport numpy as cnp
from libc.math cimport sin, cos, ceil, floor
from libc.stdio cimport sprintf, printf, puts
from libc.stdlib cimport malloc
from conehead.dda_3d_c cimport dda_3d_c, result
from conehead.vector cimport vector, vector_init, vector_append, vector_get, vector_set, vector_free

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)
def convolve_c(cnp.float64_t[:,:,:] dose_grid_terma, cnp.float64_t[:,:,:] dose_grid_dose, cnp.float64_t[:] dose_grid_dim, cnp.float64_t[:] thetas, cnp.float64_t[:] phis, kernel):

    cdef cnp.int32_t xlen = dose_grid_terma.shape[0]
    cdef cnp.int32_t ylen = dose_grid_terma.shape[1]
    cdef cnp.int32_t zlen = dose_grid_terma.shape[2]
    cdef cnp.int32_t dose_grid_shape[3]
    dose_grid_shape = [
        xlen, ylen, zlen
    ]
       
    cdef cnp.float64_t T, N, N_inv, theta_rad, phi_rad, c_t, s_t, c_p, s_p
    cdef cnp.float64_t pi = 3.14159265359
    cdef cnp.int32_t num_thetas = thetas.shape[0]
    cdef cnp.int32_t num_phis = phis.shape[0]
    cdef cnp.int32_t x, y, z, i, j, m

    cdef cnp.int32_t current_voxel[3]
    cdef cnp.float64_t direction[3]
    cdef cnp.float64_t dimensions[3] 
    dimensions = [
        dose_grid_dim[0], 
        dose_grid_dim[1],
        dose_grid_dim[2]
    ]

    cdef result r
    cdef cnp.float64_t* intersection
    cdef cnp.int32_t intersection_index
    cdef vector intersection_indices
    
    cdef char key[10]
    cdef vector* kernel_data = process_kernel(kernel, phis)
    cdef vector* current_cone_kernel_data
    cdef cnp.int32_t* v
    cdef cnp.float64_t* k1
    cdef cnp.float64_t* k2
    cdef cnp.int32_t index
    
    
    cdef char buffer[120]
    
    
    for x in range(xlen):
        for y in range(ylen):
            for z in range(zlen):

                T = dose_grid_terma[x, y, z]
                if T:

                    for i in range(num_thetas):
                        for j in range(num_phis):
                                                      
                            current_voxel = [x, y, z]

                            # Calculate direction vector
                            theta_rad = thetas[i] * pi / 180.0
                            phi_rad = phis[j] * pi / 180.0
                            c_t = cos(theta_rad)
                            s_t = sin(theta_rad)
                            c_p = cos(phi_rad)
                            s_p = sin(phi_rad)
                            direction = [
                                c_t * s_p,
                                s_t * s_p,
                                c_p
                            ]
                            N = direction[0] * direction[0] + direction[1] * direction[1] + direction[2] * direction[2]
                            N_inv = 1.0 / N
                            direction = [  # Normalise
                                direction[0] * N_inv,
                                direction[1] * N_inv,
                                direction[2] * N_inv
                            ]
                            direction = [  # Discretise
                                ceil(direction[0] * 100000) * 0.00001,
                                ceil(direction[1] * 100000) * 0.00001,
                                ceil(direction[2] * 100000) * 0.00001
                            ]
                                                                   
                            # Raytracing
                            r = dda_3d_c(
                                direction,
                                dose_grid_shape,
                                current_voxel,
                                dimensions
                            )

                            vector_init(&intersection_indices)

                            for m in range(r.intersection_t_values.size):
                                intersection = <cnp.float64_t*>vector_get(r.intersection_t_values, m)
                                intersection_index = <cnp.int32_t>floor(intersection[0] * 100.0 - 50.0)
                                intersection_index = absolute(intersection_index)
                                vector_append(&intersection_indices, copy_i(&intersection_index))

                            # sprintf(key, "%.2f", phis[j])
                            # kernel_data_ptr = get_kernel_data(kernel, phis[j])
                            current_cone_kernel_data = &kernel_data[j]            
                            for m in range(intersection_indices.size):
                                v = <cnp.int32_t*>vector_get(r.voxels_traversed, m)
                                if m == 0:                                  
                                    index1 = (<cnp.int32_t*>vector_get(&intersection_indices, m))[0]
                                    k1 = <cnp.float64_t*>vector_get(current_cone_kernel_data, index1)
                                else:
                                    index1 = (<cnp.int32_t*>vector_get(&intersection_indices, m))[0]
                                    k1 = <cnp.float64_t*>vector_get(current_cone_kernel_data, index1)
                                    index2 = (<cnp.int32_t*>vector_get(&intersection_indices, m - 1))[0]
                                    k2 = <cnp.float64_t*>vector_get(current_cone_kernel_data, index2)
                                    k1[0] = k1[0] - k2[0]

                                dose_grid_dose[v[0], v[1], v[2]] += T * k1[0]

                            vector_free(r.intersection_t_values)
                            vector_free(r.voxels_traversed)
                            vector_free(&intersection_indices)
                            
                            sprintf(buffer, "%d, %d, %d", current_voxel[0], current_voxel[1], current_voxel[2])
                            puts(buffer)

cdef cnp.int32_t absolute(cnp.int32_t value) nogil:
    if value < 0:
        return -1 * value

cdef cnp.int32_t* copy_i(cnp.int32_t* i) nogil:
    
    cdef cnp.int32_t* new_i = <cnp.int32_t*>malloc(sizeof(cnp.int32_t))
    new_i[0] = i[0]
    return new_i

cdef cnp.float64_t* copy_f(cnp.float64_t* f) nogil:
      
    cdef cnp.float64_t* new_f = <cnp.float64_t*>malloc(sizeof(cnp.float64_t))
    new_f[0] = f[0]
    return new_f

cdef vector* get_single_cone_data(kernel, key):
    cdef vector single_cone_data
    vector_init(&single_cone_data)
    cdef cnp.int32_t N = len(kernel.cumulative["{:.2f}".format(key)])
    cdef cnp.float64_t value
    for n in range(N):
        value = kernel.cumulative["{:.2f}".format(key)][n]
        vector_append(&single_cone_data, copy_f(&value))
    return &single_cone_data

cdef vector* process_kernel(kernel, phis):
    cdef vector kernel_data
    cdef vector* single_cone_data
    vector_init(&kernel_data)
    for phi in phis:
        single_cone_data = get_single_cone_data(kernel, phi)
        vector_append(&kernel_data, single_cone_data)
    return &kernel_data

UsageError: Cell magic `%%cython` not found.


In [ ]:
convolve_c(dose_grid_terma, dose_grid_dose, dose_grid_dim, thetas, phis, kernel)

In [2]:
load_ext Cython

In [8]:
%%cython

cimport numpy as cnp
from libc.stdlib cimport malloc, realloc, free, exit
from libc.stdio cimport sprintf, puts, printf

cdef struct vector:
    cnp.int32_t size
    cnp.int32_t capacity
    void** data


cdef void vector_init(vector* v) nogil:
    # Initialize size and capacity
    v.size = 0
    v.capacity = 10

    # Allocate memory for v.data
    v.data = <void**>malloc(sizeof(void*) * v.capacity)


cdef void vector_append(vector* v, void* value) nogil:
    # Make sure there's room to expand into
    vector_double_capacity_if_full(v)

    # Append the value and increment vector.size
    v.data[v.size] = value
    v.size += 1


cdef void* vector_get(vector* v, cnp.int32_t index) nogil:
    if index >= v.size or index < 0:
        printf("Index %d out of bounds for vector of size %d\n", index, v.size)
        exit(1)
    return v.data[index]


cdef void vector_set(vector* v, cnp.int32_t index, void* value) nogil:
    # Set the value at the desired index
    v.data[index] = value


cdef void vector_double_capacity_if_full(vector* v) nogil:

    cdef void** data

    if (v.size >= v.capacity):
        # double v.capacity and resize the allocated memory accordingly
        v.capacity *= 2
        #v.data = <void*>realloc(v.data, sizeof(void*) * v.capacity)

        data = <void**>realloc(v.data, sizeof(void *) * v.capacity)
        if data:
            v.data = data
        else:
            printf("Error reallocating vector memory")
            exit(1)


cdef void vector_free(vector* v) nogil:
    free(v.data)

def func():    
    cdef cnp.float64_t* floats = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 12]
    cdef vector v
    vector_init(&v)
    vector_append(&v, &floats[0])
    vector_append(&v, &floats[1])
    vector_append(&v, &floats[2])
    vector_append(&v, &floats[3])
    vector_append(&v, &floats[4])
    vector_append(&v, &floats[5])
    vector_append(&v, &floats[6])
    vector_append(&v, &floats[7])
    vector_append(&v, &floats[8])

    
    cdef char buffer[20]
    sprintf(buffer, "%d", v.capacity)
    puts(buffer)
    
    vector_append(&v, &floats[9])
    
    sprintf(buffer, "%d", v.capacity)
    puts(buffer)    
    
    vector_append(&v, &floats[10])   

    sprintf(buffer, "%d", v.capacity)
    puts(buffer)    
    
    vector_append(&v, &floats[11])   

    sprintf(buffer, "%d", v.capacity)
    puts(buffer)    
    
    vector_append(&v, &floats[12])   
    
    sprintf(buffer, "%d", v.capacity)
    puts(buffer)
    
    vector_free(&v)

In [9]:
func()

In [2]:
load_ext Cython

In [19]:
%%cython -a
import cython
cimport numpy as cnp
from conehead.vector cimport vector, vector_init, vector_append, vector_get, vector_set, vector_free
from libc.stdio cimport sprintf, puts, printf
from libc.stdlib cimport malloc

cdef struct result:
    vector* intersection_t_values
    vector* voxels_traversed

@cython.boundscheck(False)  # Deactivate bounds checking
@cython.wraparound(False)   # Deactivate negative indexing.
@cython.cdivision(True)
cdef result dda_3d_c_test(cnp.float64_t* direction, cnp.int32_t* grid_shape, cnp.int32_t* first_voxel, cnp.float64_t* voxel_size) nogil:
    """ Calculate the intersection points of a ray with a voxel grid, using a
    3D DDA algorithm.

    Parameters
    ----------
    direction : ndarray
        direction vector of the ray
    grid : ndarray
        Shape of 3D voxel grid
    first_voxel : ndarray
        Index of ray source voxel
    voxel_size : ndarray
        Size of voxel dimensions

    Returns
    -------
    intersection_t_values_mv : ndarray
        Array of t values corresponding to voxel boundary intersections
    voxels_traversed_mv : ndarray
        List of voxel indices intersected by ray
    """
    cdef cnp.int32_t step[3]
    step[0] = -1 if direction[0] < 0 else 1
    step[1] = -1 if direction[1] < 0 else 1
    step[2] = -1 if direction[2] < 0 else 1

    if direction[0] < 0:
        direction[0] = -1 * direction[0]
    if direction[1] < 0:
        direction[1] = -1 * direction[1]
    if direction[2] < 0:
        direction[2] = -1 * direction[2]

    cdef cnp.int32_t current_voxel[3]
    current_voxel[0] = first_voxel[0]
    current_voxel[1] = first_voxel[1]
    current_voxel[2] = first_voxel[2]

    cdef cnp.float64_t t[3]
    cdef cnp.float64_t delta_t[3]
    cdef cnp.float64_t big_number = 1000000000
    if direction[0] == 0.0:
        t[0] = big_number
        delta_t[0] = big_number
    else:
        t[0] = (voxel_size[0] / 2) / direction[0]
        delta_t[0] = voxel_size[0] / direction[0]
    if direction[1] == 0.0:
        t[1] = big_number
        delta_t[1] = big_number
    else:
        t[1] = (voxel_size[1] / 2) / direction[1]
        delta_t[1] = voxel_size[1] / direction[1]
    if direction[2] == 0.0:
        t[2] = big_number
        delta_t[2] = big_number
    else:
        t[2] = (voxel_size[2] / 2) / direction[2]
        delta_t[2] = voxel_size[2] / direction[2]

    cdef cnp.int32_t xmax = grid_shape[0]
    cdef cnp.int32_t ymax = grid_shape[1]
    cdef cnp.int32_t zmax = grid_shape[2]

    cdef vector voxels_traversed
    vector_init(&voxels_traversed)
    
    cdef vector intersection_t_values
    vector_init(&intersection_t_values)

    cdef cnp.int32_t v_count = 0
    cdef cnp.int32_t i_count = 0
    
            
    cdef char buffer[120]
    cdef cnp.int32_t v[3]
    cdef cnp.float64_t* tmp1
    cdef cnp.float64_t* tmp2

    while (current_voxel[0] >= 0 and current_voxel[0] < xmax and
           current_voxel[1] >= 0 and current_voxel[1] < ymax and
           current_voxel[2] >= 0 and current_voxel[2] < zmax):
             
        vector_append(&voxels_traversed, copy_v(current_voxel))
              
        if t[0] < t[1]:
            if t[0] < t[2]:       
                vector_append(&intersection_t_values, copy_t(&t[0]))
                t[0] += delta_t[0]
                current_voxel[0] = current_voxel[0] + step[0]
            else:                     
                vector_append(&intersection_t_values, copy_t(&t[1]))
                t[2] += delta_t[2]
                current_voxel[2] = current_voxel[2] + step[2]
        else:
            if t[1] < t[2]:                   
                vector_append(&intersection_t_values, copy_t(&t[1]))
                t[1] += delta_t[1]
                current_voxel[1] = current_voxel[1] + step[1]
            else:                 
                vector_append(&intersection_t_values, copy_t(&t[2]))
                t[2] += delta_t[2]
                current_voxel[2] = current_voxel[2] + step[2]     
    
    cdef result r
    r.intersection_t_values = &intersection_t_values
    r.voxels_traversed = &voxels_traversed
    
    return r

cdef cnp.float64_t* copy_t(cnp.float64_t* t) nogil:
      
    cdef cnp.float64_t* new_t = <cnp.float64_t*>malloc(sizeof(cnp.float64_t))
    new_t[0] = t[0]
    return new_t

cdef cnp.int32_t* copy_v(cnp.int32_t* v) nogil:
    
    cdef cnp.int32_t* new_v = <cnp.int32_t*>malloc(3 * sizeof(cnp.int32_t))
    new_v[0] = v[0]
    new_v[1] = v[1]
    new_v[2] = v[2]
    return new_v

def func3():
    
    cdef cnp.int32_t dose_grid_shape[3]
    dose_grid_shape = [
        5, 5, 5
    ]
    cdef cnp.int32_t current_voxel[3]
    current_voxel = [
        2, 2, 2
    ]
    cdef cnp.float64_t direction[3]
    direction = [
        1.0, 0.0, 0.0
    ]
    cdef cnp.float64_t dimensions[3] 
    dimensions = [
        2.0, 2.0, 2.0
    ]

    cdef result r2 

    r2 = dda_3d_c_test(
        direction,
        dose_grid_shape,
        current_voxel,
        dimensions
    )
    
    cdef char buffer[120]
    cdef cnp.int32_t v[3]
    cdef cnp.int32_t s, i
    cdef cnp.float64_t t
    
    #sprintf(buffer, "%d", r2.voxels_traversed.size)
    #puts(buffer)
    for i in range(r2.voxels_traversed.size):
        v[0] = (<cnp.int32_t*>vector_get(r2.voxels_traversed, i))[0]
        v[1] = (<cnp.int32_t*>vector_get(r2.voxels_traversed, i))[1]
        v[2] = (<cnp.int32_t*>vector_get(r2.voxels_traversed, i))[2]
        sprintf(buffer, "%d, %d, %d", v[0], v[1], v[2])
        puts(buffer)
        
    for i in range(r2.intersection_t_values.size):
        t = (<cnp.float64_t*>vector_get(r2.intersection_t_values, i))[0]
        sprintf(buffer, "%f", t)
        puts(buffer)
      

In [18]:
func3()